# Exercise 9

## Mashable news stories analysis

Predicting if a news story is going to be popular


In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

url = 'https://raw.githubusercontent.com/albahnsen/PracticalMachineLearningClass/master/datasets/mashable.csv'
train_df = pd.read_csv(url, index_col=0)
train_df.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,Popular
0,http://mashable.com/2014/12/10/cia-torture-rep...,28.0,9.0,188.0,0.732620,1.0,0.844262,5.0,1.0,1.0,...,0.200000,0.80,-0.487500,-0.60,-0.250000,0.9,0.8,0.4,0.8,1
1,http://mashable.com/2013/10/18/bitlock-kicksta...,447.0,7.0,297.0,0.653199,1.0,0.815789,9.0,4.0,1.0,...,0.160000,0.50,-0.135340,-0.40,-0.050000,0.1,-0.1,0.4,0.1,0
2,http://mashable.com/2013/07/24/google-glass-po...,533.0,11.0,181.0,0.660377,1.0,0.775701,4.0,3.0,1.0,...,0.136364,1.00,0.000000,0.00,0.000000,0.3,1.0,0.2,1.0,0
3,http://mashable.com/2013/11/21/these-are-the-m...,413.0,12.0,781.0,0.497409,1.0,0.677350,10.0,3.0,1.0,...,0.100000,1.00,-0.195701,-0.40,-0.071429,0.0,0.0,0.5,0.0,0
4,http://mashable.com/2014/02/11/parking-ticket-...,331.0,8.0,177.0,0.685714,1.0,0.830357,3.0,2.0,1.0,...,0.100000,0.55,-0.175000,-0.25,-0.100000,0.0,0.0,0.5,0.0,0


In [2]:
train_df.shape

(6000, 61)

In [3]:
X = train_df.drop(['url', 'Popular'], axis=1)
y = train_df['Popular']

In [4]:
y.mean()

0.5

In [5]:
# train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Exercise 9.1

Estimate a Decision Tree Classifier and a Logistic Regresion

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

treeclf = DecisionTreeClassifier(max_depth=3, random_state=1)
treeclf.fit(X_train, y_train)
y_pred = treeclf.predict(X_test)

print('Accuracy ',metrics.accuracy_score(y_test, y_pred))
print('F1 score ',metrics.f1_score(y_test, y_pred))

Accuracy  0.6413333333333333
F1 score  0.6072992700729928


In [7]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

from sklearn import metrics
print('Accuracy ',metrics.accuracy_score(y_test, y_pred))
print('F1 score ',metrics.f1_score(y_test, y_pred))

Accuracy  0.6273333333333333
F1 score  0.6104529616724739


# Exercise 9.2

Estimate 300 bagged samples

Estimate the following set of classifiers:

* 100 Decision Trees where max_depth=None
* 100 Decision Trees where max_depth=2
* 100 Logistic Regressions

In [8]:
np.random.seed(123)
n_samples = X_train.shape[0]
n_B = 300

samples = [np.random.choice(a=n_samples, size=n_samples, replace=True) for _ in range(1, n_B +1 )]
len(samples)

300

In [9]:
n_estimators = 300

np.random.seed(123) 
seeds = np.random.randint(1, 10000, size=n_estimators)

trees = {}
for i in range(100):
    trees[i] = DecisionTreeClassifier(max_depth=None, random_state=seeds[i])
    trees[i].fit(X_train.iloc[samples[i]], y_train.iloc[samples[i]])

In [10]:
for i in range(100,200):
    trees[i] = DecisionTreeClassifier(max_depth=2, random_state=seeds[i])
    trees[i].fit(X_train.iloc[samples[i]], y_train.iloc[samples[i]])


In [11]:
for i in range(200,300):
    trees[i] = LogisticRegression(random_state=seeds[i])
    trees[i].fit(X_train.iloc[samples[i]], y_train.iloc[samples[i]])


# Exercise 9.3

Ensemble using majority voting

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [12]:
y_pred_df = pd.DataFrame(index=X_test.index, columns=list(range(n_estimators)))
for i in range(n_estimators):
    y_pred_df.iloc[:, i] = trees[i].predict(X_test)

y_pred_df.head()


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
1483,1,1,0,0,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2185,1,1,1,1,0,1,1,1,1,0,...,1,1,1,1,1,1,1,1,1,1
2520,1,0,0,0,1,0,1,1,0,0,...,1,1,1,1,1,1,1,1,1,1
3721,1,1,1,0,1,0,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3727,0,0,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
y_pred = (y_pred_df.sum(axis=1) >= (n_estimators / 2)).astype(np.int)
print('F1_score ', metrics.f1_score(y_pred, y_test))
print('Accuracy ', metrics.accuracy_score(y_pred, y_test))

F1_score  0.6253443526170799
Accuracy  0.6373333333333333


# Exercise 9.4

Estimate te probability as %models that predict positive

Modify the probability threshold and select the one that maximizes the F1-Score

In [14]:
y_pred=((y_pred_df.sum(axis=1)/n_estimators) >= 0.5).astype(np.int)
y_pred.head()

y_pred.value_counts().to_frame('count').assign(percentage = lambda x: x/x.sum())

,count,percentage
0,804,0.536
1,696,0.464


In [15]:
def frange(start, stop, step):
    i = start
    while i < stop:
        yield i
        i += step

In [16]:
for i in frange (0, 1, 0.01):
    y_pred=((y_pred_df.sum(axis=1)/n_estimators) >= i).astype(np.int)
    print(i)
    print('F1_score ', metrics.f1_score(y_pred, y_test))


0
F1_score  0.6702127659574468
0.01
F1_score  0.6705099778270509
0.02
F1_score  0.6714095153401511
0.03
F1_score  0.6738159070598748
0.04
F1_score  0.6753246753246753
0.05
F1_score  0.6780575539568344
0.060000000000000005
F1_score  0.6818596171376481
0.07
F1_score  0.6875287620800737
0.08
F1_score  0.6884328358208955
0.09
F1_score  0.6948669201520913
0.09999999999999999
F1_score  0.6982134234669242
0.10999999999999999
F1_score  0.7012732615083251
0.11999999999999998
F1_score  0.7058823529411765
0.12999999999999998
F1_score  0.7062404870624048
0.13999999999999999
F1_score  0.7066735644076564
0.15
F1_score  0.7076923076923076
0.16
F1_score  0.700650759219089
0.17
F1_score  0.6940588561910049
0.18000000000000002
F1_score  0.6913439635535308
0.19000000000000003
F1_score  0.6858789625360231
0.20000000000000004
F1_score  0.67953216374269
0.21000000000000005
F1_score  0.6784452296819788
0.22000000000000006
F1_score  0.6765578635014837
0.23000000000000007
F1_score  0.6766467065868265
0.2400000

Se escoge el umbral de 0.15 pues es el que maximiza el F1_score.

# Exercise 9.5

Ensemble using weighted voting using the oob_error

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [17]:
samples_oob = []
for sample in samples:
    samples_oob.append(sorted(set(range(n_samples)) - set(sample)))

errors = np.zeros(n_estimators)

for i in range(n_estimators):
    y_pred_ = trees[i].predict(X_train.iloc[samples_oob[i]])
    errors[i] = 1 - metrics.accuracy_score(y_train.iloc[samples_oob[i]], y_pred_)
    
metrics.f1_score(y_pred, y_test), metrics.accuracy_score(y_pred, y_test)

(0.0, 0.496)

# Exercise 9.6

Estimate te probability of the weighted voting

Modify the probability threshold and select the one that maximizes the F1-Score

In [18]:
alpha = (1 - errors) / (1 - errors).sum()
weighted_sum_1 = ((y_pred_df) * alpha).sum(axis=1)
y_pred = (weighted_sum_1 >= 0.5).astype(np.int)
y_pred.head()

for i in frange (0, 1, 0.01):
    y_pred = (weighted_sum_1 >= i).astype(np.int)
    print(i)
    print('F1_score ', metrics.f1_score(y_pred, y_test))

0
F1_score  0.6702127659574468
0.01
F1_score  0.6705099778270509
0.02
F1_score  0.6723063223508459
0.03
F1_score  0.6741171211443898
0.04
F1_score  0.6765365634813818
0.05
F1_score  0.6787330316742081
0.060000000000000005
F1_score  0.6852954649564819
0.07
F1_score  0.6911286576869484
0.08
F1_score  0.6931442080378252
0.09
F1_score  0.6981222917669715
0.09999999999999999
F1_score  0.7018572825024438
0.10999999999999999
F1_score  0.7055306427503737
0.11999999999999998
F1_score  0.7058823529411765
0.12999999999999998
F1_score  0.7067357512953368
0.13999999999999999
F1_score  0.7060702875399362
0.15
F1_score  0.7008174386920981
0.16
F1_score  0.6937639198218263
0.17
F1_score  0.6905574516496018
0.18000000000000002
F1_score  0.6847575057736721
0.19000000000000003
F1_score  0.6799065420560747
0.20000000000000004
F1_score  0.6768687463213655
0.21000000000000005
F1_score  0.6773428232502966
0.22000000000000006
F1_score  0.6750298685782556
0.23000000000000007
F1_score  0.6762936221419975
0.2400

En este caso se elige el 0.13 como umbral que maximiza el F1.

# Exercise 9.7

Estimate a logistic regression using as input the estimated classifiers

Modify the probability threshold such that maximizes the F1-Score

In [19]:
X_train_2 = pd.DataFrame(index=X_train.index, columns=list(range(n_estimators)))

for i in range(n_estimators):
    X_train_2[i] = trees[i].predict(X_train)
    
from sklearn.linear_model import LogisticRegressionCV
lr = LogisticRegressionCV(cv = 5, random_state=123)
lr.fit(X_train_2, y_train)

y_pred = lr.predict(y_pred_df)

print(metrics.f1_score(y_pred, y_test), metrics.accuracy_score(y_pred, y_test))
y_pred


0.6425755584756899 0.6373333333333333


array([1, 1, 1, ..., 1, 0, 0], dtype=int64)

In [27]:
for i in frange (0, 1, 0.01):
    y_pred = np.where(lr.predict_proba(y_pred_df)[:,1] >= i, 1, 0)
    print(([ i, metrics.f1_score(y_pred, y_test)]))


[0, 0.6702127659574468]
[0.01, 0.6702127659574468]
[0.02, 0.6702127659574468]
[0.03, 0.6714095153401511]
[0.04, 0.6741171211443898]
[0.05, 0.6762331838565023]
[0.060000000000000005, 0.6792792792792792]
[0.07, 0.6805807622504538]
[0.08, 0.6834631241410902]
[0.09, 0.6870937790157846]
[0.09999999999999999, 0.6877912395153775]
[0.10999999999999999, 0.6890042472864558]
[0.11999999999999998, 0.6913932477413219]
[0.12999999999999998, 0.6925659472422062]
[0.13999999999999999, 0.6956102267245537]
[0.15, 0.6971762414800389]
[0.16, 0.7]
[0.17, 0.70262246412667]
[0.18000000000000002, 0.7048853439680957]
[0.19000000000000003, 0.7040560841261893]
[0.20000000000000004, 0.7055863110216406]
[0.21000000000000005, 0.7070707070707071]
[0.22000000000000006, 0.7080366225839267]
[0.23000000000000007, 0.7081192189105858]
[0.24000000000000007, 0.708612686952037]
[0.25000000000000006, 0.7088738972496108]
[0.26000000000000006, 0.7078534031413612]
[0.2700000000000001, 0.7056345444971037]
[0.2800000000000001, 0.70

El umbral que maximiza el f1_score es 0.25 con un f1_score de 0,7088.